# Part 0:
## Import necessary packages

In [13]:
import math
import os
import subprocess
import time
import yaml

import gurobipy as gp
import hmmlearn.hmm as hmm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf

from datetime import datetime
from gurobipy import *
from scipy.stats import skew

# Part 1:
## Get the Current Market Regime

In [ ]:
datadir = os.path.join(os.getcwd(),'..','data')

In [6]:
fpath = os.path.join(os.getcwd(), 'hmm.R')
run_time = datetime.now()
subprocess.call(["C:/Program Files/R/R-3.6.2/bin/Rscript.exe", fpath])

# end_of_trading_today = datetime.strptime(str(datetime.date(datetime.now()))+ " 16:00", "%Y-%m-%d %H:%M")

# if run_time > end_of_trading_today:
#     # If the current day's trading hours have ended, caluclate the regime and record it as the past regime
    
#     subprocess.call(["C:/Program Files/R/R-3.6.2/bin/Rscript.exe", fpath])
#     data_dict = [{'Prior_Regime':current_regime},{'Prior_Date':str(datetime.date(datetime.now()))}]
    
#     with open('prior_regime.yaml', 'w') as file:
#         documents = yaml.dump(dict_file, file)
        
        
# elif run_time < end_of_trading_today:
    
#     with open('prior_regime.yaml') as file:
#         documents = yaml.full_load(file)
#         current_regime = documents['Prior_Regime']

0

In [7]:
regime_file = os.path.join(datadir, 'pprobsGSPC.csv')
regimes = pd.read_csv(regime_file, index_col=0)
regimes_list = list(regimes.State_Name)
prior_regime = regimes_list[-2]
current_regime = regimes_list[-1]
change_regime = current_regime != prior_regime
current_regime

'Level3'

# Part 2:
## Get the current market regime

In [2]:
sp500_price_file = os.path.join(datadir, 'sp500_individual_prices_05-08.csv')
sp500_returns_file = os.path.join(datadir, 'sp500_individual_returns_05-08.csv')

In [3]:
price_df = pd.read_csv(sp500_price_file, index_col=0)
returns_df = pd.read_csv(sp500_returns_file, index_col=0)

In [4]:
price_df.tail()

,^GSPC,UPRO,SSO,GLD,TLT,MMM,ABT,ABBV,ABMD,ACN,...,WYNN,XEL,XRX,XLNX,XYL,YUM,ZBRA,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2020-05-04,2842.74,35.23,106.09,160.34,167.16,148.40,89.83,81.86,181.00,179.53,...,80.45,62.39,16.82,83.24,68.55,83.09,228.77,114.07,30.32,127.44
2020-05-05,2868.44,36.17,108.01,161.02,166.09,147.43,93.03,85.37,186.22,181.62,...,79.67,63.28,17.03,83.50,65.05,83.28,233.15,118.02,29.84,131.30
2020-05-06,2848.42,35.45,106.53,158.95,163.41,146.20,91.08,85.42,186.91,180.38,...,79.25,61.22,16.70,85.24,63.06,83.17,232.42,115.88,29.54,125.69
2020-05-07,2881.19,36.71,109.08,161.39,166.14,145.74,93.99,84.22,188.33,187.11,...,79.60,59.96,17.33,84.43,63.55,84.60,233.86,119.40,29.57,124.24
2020-05-08,2929.80,38.51,112.69,160.42,163.98,148.51,93.70,83.96,187.07,188.56,...,86.94,60.31,18.13,86.86,63.00,86.18,242.92,121.86,30.72,125.47


In [5]:
returns_df.tail()

,^GSPC,UPRO,SSO,GLD,TLT,MMM,ABT,ABBV,ABMD,ACN,...,WYNN,XEL,XRX,XLNX,XYL,YUM,ZBRA,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2020-05-04,0.424982,1.061388,0.645100,0.350482,-0.470378,-0.134590,-0.022259,-1.183003,-1.357022,-0.310956,...,-0.470122,0.337729,-1.233118,0.301241,-1.067975,-0.859086,1.427621,-1.875269,-0.752864,-0.070572
2020-05-05,0.904057,2.668181,1.809784,0.424099,-0.640105,-0.653639,3.562284,4.287808,2.883978,1.164151,...,-0.969546,1.426511,1.248514,0.312350,-5.105762,0.228668,1.914587,3.462786,-1.583113,3.028876
2020-05-06,-0.697940,-1.990600,-1.370243,-1.285555,-1.613583,-0.834294,-2.096098,0.058569,0.370529,-0.682744,...,-0.527175,-3.255373,-1.937757,2.083832,-3.059185,-0.132085,-0.313103,-1.813252,-1.005362,-4.272658
2020-05-07,1.150462,3.554302,2.393692,1.535074,1.670644,-0.314637,3.194993,-1.404823,0.759724,3.731012,...,0.441640,-2.058151,3.772455,-0.950258,0.777038,1.719370,0.619568,3.037625,0.101557,-1.153632
2020-05-08,1.687150,4.903296,3.309498,-0.601029,-1.300108,1.900645,-0.308543,-0.308715,-0.669038,0.774945,...,9.221106,0.583722,4.616272,2.878124,-0.865460,1.867612,3.874113,2.060302,3.889077,0.990019


In [8]:
num_states = len(regimes_list)
returns_df = returns_df.iloc[-num_states:,:]
price_df = price_df.iloc[-num_states:,:]
returns_df["State_Names"] = regimes_list
price_df["State_Names"] = regimes_list
returns_df

,^GSPC,UPRO,SSO,GLD,TLT,MMM,ABT,ABBV,ABMD,ACN,...,XEL,XRX,XLNX,XYL,YUM,ZBRA,ZBH,ZION,ZTS,State_Names
Date,,,,,,,,,,,,,,,,,,,,,
1928-01-27,0.340329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Level2
1928-01-30,-1.130582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Level2
1928-01-31,0.457404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Level2
1928-02-01,-0.227661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Level1
1928-02-02,0.570451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Level1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-04,0.424982,1.061388,0.645100,0.350482,-0.470378,-0.134590,-0.022259,-1.183003,-1.357022,-0.310956,...,0.337729,-1.233118,0.301241,-1.067975,-0.859086,1.427621,-1.875269,-0.752864,-0.070572,Level3
2020-05-05,0.904057,2.668181,1.809784,0.424099,-0.640105,-0.653639,3.562284,4.287808,2.883978,1.164151,...,1.426511,1.248514,0.312350,-5.105762,0.228668,1.914587,3.462786,-1.583113,3.028876,Level3
2020-05-06,-0.697940,-1.990600,-1.370243,-1.285555,-1.613583,-0.834294,-2.096098,0.058569,0.370529,-0.682744,...,-3.255373,-1.937757,2.083832,-3.059185,-0.132085,-0.313103,-1.813252,-1.005362,-4.272658,Level3


# Part 3:
## Calculate the Maximum Skew Portfolio

In [9]:
# Invoke Gurobi to solve the LP
def solve_lp(methnum, annualized_r, sigma, inflation_factor, regime):

    # Model setup
    gm = gp.Model("Konno")

    # Create the data
    mu, B, T, n, returns, cnames = get_data(regime)
    
    r_expected = (((1+(annualized_r/100))**(1/252))-1)*100 # Set the expected return here.  Asssumes 252 trading days per year
    print(r_expected)
    rj = mu # means are denoted by r in Konno's paper
    alpha = 1 # Set value for alpha
    time_inverse = 1/(T-1) # Constant for minimization of objective function
    
    w = (T-1)*np.sqrt(2/math.pi)*sigma*inflation_factor
    
    rho_1 = r_expected-1
    rho_2 = r_expected-2
    
    min_prop = 0.04
    max_prop =0.2

    # --------------------------------------------------------------------------
    # IMPLEMENT LP Formulation Here
    # --------------------------------------------------------------------------
    
    # Add vars
    u = gm.addVars(T)  # For calulating Skewness
    v = gm.addVars(T)  # For calculating Skewness
    x = gm.addVars(n)  # Weights of each asset
    XI = gm.addVars(T) # For constraining
    H = gm.addVars(T)  # For constraining
    
    # Set the Objective
    gm.setObjective(time_inverse*(quicksum(u[t] for t in range(T)) + alpha*quicksum(v[t] for t in range(T))), GRB.MINIMIZE)
    
    y = gm.addVars(n, vtype=GRB.BINARY)
    gm.addConstrs(y[j] >= x[j] for j in range(n))
    gm.addConstrs(min_prop*y[j] <= x[j] for j in range(n))
    gm.addConstrs(max_prop*y[j] >= x[j] for j in range(n))
    
    # Add constraints
    gm.addConstrs((u[t]+quicksum(returns[t,j]*x[j] for j in range(n))) >= rho_1 for t in range(T))
    gm.addConstrs((v[t]+quicksum(returns[t,j]*x[j] for j in range(n))) >= rho_2 for t in range(T))
    
    gm.addConstrs(XI[t]-H[t]-quicksum(returns[t,j]*x[j] for j in range(n)) >= r_expected for t in range(T))
    gm.addConstr(quicksum(rj[j]*x[j] for j in range(n)) >= r_expected)

    gm.addConstr(quicksum(XI[t]+H[t] for t in range(T)) <= w)
    gm.addConstr(quicksum(x[j] for j in range(n)) == 1)
    gm.addConstrs(x[j] >= 0 for j in range(n))
    gm.addConstrs(u[t] >= 0 for t in range(T))
    gm.addConstrs(v[t] >= 0 for t in range(T))
    gm.addConstrs(XI[t] >= 0 for t in range(T))
    gm.addConstrs(H[t] >= 0 for t in range(T))       
    
    # Solve the model
    meth = methnum
    gm.setParam('Method', meth)
    gm.setParam('OutputFlag', 0)
    gm.update()
    
    gm.setParam('LogToConsole', 1)
    gm.setParam('LogFile', "method_{}.log".format(meth))
    
    gm.optimize()
    
    
    weights = []
    for k in x.keys():
        weights.append(x[k].x)
    weights = np.array(weights)
    #print(weights)
    print(np.mean(returns.dot(weights)))
    print(calc_geom_mean(returns.dot(weights)))
    print("")
    print(np.std(returns.dot(weights)))
    print(skew(returns.dot(weights)))
    
    return(gm, weights, skew(returns.dot(weights)), cnames)

# Generate the data
def get_data(regime):
    B = 100000
    
    #returns = returns_df.iloc[-1500:,:]
    filtered_returns = returns_df[returns_df.State_Names == regime]
    returns = filtered_returns.iloc[:,:-1]    # remove regimes names column
    enough_returns_level = np.floor(504*1) # Returns must have a minumum of X*252 days of returns in the given regime
    enough_returns_list = [i for i, _ in enumerate(returns.columns) if returns.iloc[:,i].count() >= enough_returns_level]
    returns = returns.iloc[:,enough_returns_list]
    cnames = returns.columns
    
    returns = returns.to_numpy()
    returns = returns[~np.isnan(returns).any(axis=1)]  # Drop any rows containing nan
    returns = returns[~np.isinf(returns).any(axis=1)]  # Drop any rows containing infinity
    T = returns.shape[0]
    num_assets = returns.shape[1]
    mu_geom = calc_geom_means(returns)
    #mu = np.mean(returns, axis=0)
    #print(mu_geom)
    print(returns.shape)
    
    return(mu_geom, B, T, num_assets, returns, cnames)

In [10]:
def calc_geom_mean(ret):
    scaled_col = 1+(ret/100)
    geom_avg = np.prod(scaled_col)**(1/len(ret))
    rescaled_geom_avg = (geom_avg-1)*100
    return(rescaled_geom_avg)

def calc_geom_means(rets):
    means = []
    for i in range(rets.shape[1]):
        scaled_col = 1+(rets[:,i]/100)
        geom_avg = np.prod(scaled_col)**(1/rets.shape[0])
        rescaled_geom_avg = (geom_avg-1)*100
        means.append(rescaled_geom_avg)
    return(np.array(means))

In [ ]:
start_time = time.time()
out, weights, skw, cn = solve_lp(2,30,1,.9, regime="Level4")
time.time()-start_time
purchased = np.where(final_weights > 0.01)[0]
buylist = list(cn[purchased])
print(buylist)

In [ ]:
# geom mean return at exp = 10
ret = 0.07020951669960418
(1+ret/100)**252

In [ ]:
buylist

In [ ]:
# arith mean return at exp = 10
ret = 0.07020951669960418
(1+ret/100)**252

In [ ]:
buylist

In [ ]:
#current_regime = "Level3" 

In [11]:
final_output = None
final_weights = None
final_skew = -np.inf
final_annualized_r = -np.inf
final_sd_inflation = None


min_allowable_skew = 0.1
min_sd_inflation = 0.2
max_sd_inflation = 1.1
min_allowable_annualized_r = 20

fin_flag = False
for annualized_r in [30,20,19,18,17,16,15,14,13,12,11,10]:
    
    if fin_flag:
        break
    
    for inflation_factor in np.arange(min_sd_inflation+0.6, max_sd_inflation, 0.1):
        print('Annualized r {} vs Final r {}'.format(annualized_r, final_annualized_r))
            
        try:
            out, weights, skw, cn = solve_lp(methnum=2,annualized_r=annualized_r,sigma=1,inflation_factor=inflation_factor, regime=current_regime)
            if (annualized_r >= final_annualized_r) & (skw >= min_allowable_skew):
                final_output = out
                final_weights = weights
                final_skew = skw
                final_annualized_r = annualized_r
                final_sd_inflation = inflation_factor
                fin_flag = True
                break
        except AttributeError:
            print("Infeasible on anr {}, inflf {}".format(annualized_r, inflation_factor))
            
            
print("finished")

Annualized r 30 vs Final r -inf
Using license file C:\Users\richa\gurobi.lic
Academic license - for non-commercial use only
(277, 498)
0.1041670195529365
Changed value of parameter Method to 2
   Prev: -1  Min: -1  Max: 5  Default: -1
0.12042849424355023
0.11929645838506531

0.4761243912590216
0.028599384735203136
Annualized r 30 vs Final r -inf
(277, 498)
0.1041670195529365
Changed value of parameter Method to 2
   Prev: -1  Min: -1  Max: 5  Default: -1
0.12180386641358347
0.12050417407059033

0.5110446107890874
1.0544948641575642
finished


In [12]:
print("Annualized R: {}%".format(final_annualized_r))
print("Final Skew: {}".format(final_skew))
print("SDIF Level: {}".format(final_sd_inflation))
purchased = np.where(final_weights > 0.01)[0]
buylist = list(cn[purchased])
print(buylist)
print(final_weights[purchased])

Annualized R: 30%
Final Skew: 1.0544948641575642
SDIF Level: 0.9
['GLD', 'TLT', 'AMZN', 'AMCR', 'CHTR', 'CHD', 'CTXS', 'CPRT', 'CMI', 'ETN', 'EL', 'MKTX', 'NLOK', 'ZION']
[0.2        0.2        0.05637776 0.04       0.04       0.06316921
 0.08884282 0.04       0.04       0.04       0.04       0.04
 0.07161022 0.04      ]


In [14]:
balance = 780.58+741.95

values = balance*final_weights[purchased]

for ticker, val in zip(buylist, values):
    print(ticker)
    print(val)
    shares = round(val/price_df.loc[:,ticker][-1],2)
    print(shares)
    print("")

GLD
304.50600000000003
1.9

TLT
304.50600000000003
1.86

AMZN
85.83682741083804
0.04

AMCR
60.90120000000001
6.79

CHTR
60.90120000000005
0.12

CHD
96.177015380424
1.31

CTXS
135.2658534644206
0.9

CPRT
60.90120000000001
0.73

CMI
60.90120000000001
0.38

ETN
60.90120000000001
0.74

EL
60.90120000000001
0.35

MKTX
60.90120000000001
0.13

NLOK
109.0287037443176
5.12

ZION
60.90120000000001
1.98



In [ ]:
filtered_returns = returns_df[returns_df.State_Names == "Level3"]
returns = filtered_returns.iloc[:,:-1]    # remove regimes names column
enough_returns_level = np.floor(252*2) # Returns must have a minumum of X*252 days of returns in the given regime
enough_returns_list = [i for i, _ in enumerate(returns.columns) if returns.iloc[:,i].count() >= enough_returns_level]
returns = returns.iloc[:,enough_returns_list]

returns = returns.to_numpy()
returns = returns[~np.isnan(returns).any(axis=1)]  # Drop any rows containing nan
returns = returns[~np.isinf(returns).any(axis=1)]  # Drop any rows containing infinity
sns.distplot(returns.dot(final_weights))

In [ ]:
reduced_returns = returns[:,purchased]
reduced_returns = 1+(reduced_returns/100)
fact = final_weights[purchased]/np.min(final_weights[purchased])

result = []

for i,f in enumerate(fact):
    result.append(np.prod(reduced_returns[:,i])**f)

result

In [ ]:
filtered_returns = returns_df[returns_df.State_Names == "Level4"]
returns = filtered_returns.iloc[:,:-1]    # remove regimes names column
enough_returns_level = np.floor(252*1) # Returns must have a minumum of X*252 days of returns in the given regime
enough_returns_list = [i for i, _ in enumerate(returns.columns) if returns.iloc[:,i].count() >= enough_returns_level]
returns = returns.iloc[:,enough_returns_list]
cnames = returns.columns
returns = returns.to_numpy()
returns = returns[~np.isnan(returns).any(axis=1)]  # Drop any rows containing nan
returns = returns[~np.isinf(returns).any(axis=1)]  # Drop any rows containing infinity
rets = pd.DataFrame(returns)
rets.columns = cnames
rets.head()

In [ ]:
purchased

In [ ]:
returns[:2,:5]

In [ ]:
Level1
Annualized R: 30%
Final Skew: 0.16600037460262837
SDIF Level: 0.5000000000000001
['TLT', 'ARE', 'AMZN', 'AMT', 'AIZ', 'CAT', 'EVRG', 'FAST', 'JBHT', 'KHC', 'MKC', 'NEM']
[0.2        0.09373924 0.12308992 0.10992268 0.04       0.1554293
 0.04       0.06762527 0.04141588 0.04202133 0.04       0.04675638]

In [ ]:
Level2
Annualized R: 30%
Final Skew: 0.2073998325094798
SDIF Level: 0.6000000000000001
['GLD', 'TLT', 'ADI', 'BAC', 'CAH', 'DLR', 'ECL', 'GPN', 'HIG', 'HLT', 'ISRG', 'MLM', 'NUE', 'ORLY', 'UAA']
[0.07322196 0.18065759 0.04       0.07968345 0.0541644  0.04
 0.07349316 0.04       0.12884418 0.05960961 0.04       0.063432
 0.04689366 0.04       0.04      ]

In [ ]:
Level3
Annualized R: 19%
Final Skew: 0.11494147409988621
SDIF Level: 0.6000000000000001
['ABMD', 'BR', 'CE', 'CNP', 'ETN', 'ES', 'XOM', 'FIS', 'FCX', 'GIS', 'GPC', 'JKHY', 'LYB', 'NFLX', 'PPG']
[0.04       0.06033826 0.05602146 0.04058627 0.11644978 0.04
 0.04821402 0.04       0.04       0.04       0.11449482 0.11579815
 0.04915008 0.1550775  0.04386966]

In [ ]:
Level4


In [37]:
now = datetime.now()
then = datetime.strptime('2020-05-12 16:00', "%Y-%m-%d %H:%M")
print(now)
print(then)
now>then
type(datetime.date(datetime.now()).day)

2020-05-12 08:34:42.595504
2020-05-12 16:00:00


int